<a href="https://colab.research.google.com/github/Sebastiantenb/word-matching-fuzzywuzzy/blob/main/Word_Matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
import numpy as np

In [5]:
list_base_word = ["Apple", "Pear", "Banana", "Orange", "Peach"]
list_match_word = ["apple", "Bear", "Bannana", "Orange", "Peaches", "Banaan"]

In [6]:
df_base_word = pd.DataFrame(list_base_word, columns = ["base_word"])
df_match_word = pd.DataFrame(list_match_word, columns = ["match_word"])

In [7]:
df_base_word

,base_word
0,Apple
1,Pear
2,Banana
3,Orange
4,Peach


In [8]:
df_match_word

,match_word
0,apple
1,Bear
2,Bannana
3,Orange
4,Peaches
5,Banaan


In [9]:
import sys
!{sys.executable} -m pip install fuzzywuzzy

In [10]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [11]:
list_matches = process.extractOne("Banana", df_match_word["match_word"], scorer=fuzz.ratio)
list_matches

('Bannana', 92, 2)

In [12]:
list_matches[0][1]

'a'

In [27]:
df_base_word['matched_word'] = df_base_word["base_word"].map(lambda x: process.extractOne(x, df_match_word["match_word"])[0])
df_base_word['match_ratio'] = df_base_word["base_word"].map(lambda x: process.extractOne(x, df_match_word["match_word"])[1])

In [14]:
df_base_word.head(10)

,base_word,matched_word,match_ratio
0,Apple,apple,100
1,Pear,Bear,75
2,Banana,Bannana,92
3,Orange,Orange,100
4,Peach,Peaches,83


In [52]:
print(fuzz.ratio("Apple", "apple"))

80


In [15]:
print(df_base_word["base_word"].map(lambda x: process.extractOne(x, df_match_word["match_word"])[1]))

0    100
1     75
2     92
3    100
4     83
Name: base_word, dtype: int64


In [53]:
# Create a function that takes two lists of strings for matching
def match_name(name, list_names, min_score=0):
    # -1 score incase we don't get any matches
    max_score = -1
    # Returning empty name for no match as well
    max_name = ""
    # Iterating over all names in the second list
    for name2 in list_names:
        #Finding fuzzy match score
        score = fuzz.ratio(str.lower(name), str.lower(name2))
        # Checking if we are above our threshold and have a better score
        if (score > min_score) & (score > max_score):
            max_name = name2
            max_score = score
    return (max_name, max_score)

In [67]:
# List for dicts for easy DataFrame creation
dict_list = []
# iterating over df with more strains
for name in df_base_word.base_word:
    # Use our method to find best match, we can set a threshold here
    match = match_name(name, df_match_word.match_word, 85)
    
    # New dict for storing data
    dict_ = {}
    dict_.update({"strain" : name})
    dict_.update({"name" : match[0]})
    dict_list.append(dict_)
    
df_matched_table = pd.DataFrame(dict_list)

In [68]:
df_matched_table

,strain,name
0,Apple,apple
1,Pear,
2,Banana,Bannana
3,Orange,Orange
4,Peach,
